### Modelling Notebook

In [63]:
# importing external libraries
from pathlib import Path
import os
import pandas as pd
import pickle
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn import set_config
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
set_config(transform_output = "pandas")

# Importing function to load data

# Making sure any changes are instantly added
%load_ext autoreload
%autoreload 2

from Modules.load_data import load_data
from Modules.preprocessing import missing_summary, merge_dfs, dollar_to_int, find_unique_values
from Modules.plotting import Plotter
from Modules.transforming import *

# Importing Pipelines
from Pipelines.Pipeline1 import Pipeline_for_exploration, Pipeline1, general_transformation_pipeline
from Pipelines.Pipeline2 import Pipeline_for_exploration2, Pipeline2, general_transformation_pipeline2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Obtaining absolute path to data folder

data_folder = str(Path(os.getcwd()) / "data")

In [19]:
# Loading the data from pickle

merged_df = pd.read_pickle(data_folder + "/merged_data.pkl")

In [20]:
# Applying the general transformation pipeline

merged_df1 = general_transformation_pipeline.fit_transform(merged_df)

/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(X).to_frame()
/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(X).to_frame()


In [64]:
merged_df2 = general_transformation_pipeline2.fit_transform(merged_df)

/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(X).to_frame()
/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(X).to_frame()


In [65]:
merged_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102348 entries, 0 to 102347
Data columns (total 33 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   has_chip               102348 non-null  int64         
 1   card_on_dark_web       102348 non-null  int64         
 2   amount                 102348 non-null  int64         
 3   credit_limit           102348 non-null  int64         
 4   date                   102348 non-null  datetime64[ns]
 5   expires                102348 non-null  datetime64[ns]
 6   acct_open_date         102348 non-null  datetime64[ns]
 7   client_id              102348 non-null  int64         
 8   card_id                102348 non-null  int64         
 9   use_chip               102348 non-null  object        
 10  merchant_id            102348 non-null  int64         
 11  merchant_city          102348 non-null  object        
 12  merchant_state         102348 non-null  obje

In [5]:
# Setting the target column

y = merged_df["is_fraud"]

In [6]:
# Test Train Split

X_train, X_test, y_train, y_test = train_test_split(merged_df, y, test_size=0.25, random_state=42)

In [7]:
# Applying Pipeline1

X_train = Pipeline1.fit_transform(X_train)
X_test = Pipeline1.transform(X_test)

In [8]:
# Checkiing the proportion of positive values

print(f"% of fraudulent transactions in y_train: {y_train.mean()}")
print(f"% of fraudulent transactions in y_test: {y_test.mean()}")


% of fraudulent transactions in y_train: 0.1312906293560532
% of fraudulent transactions in y_test: 0.12717395552428967


In [9]:
# Setting params to do grid search over
numerical_parameter_grid = np.linspace(0,1,10)

params = [{
    "penalty": [None, "elasticnet"],
    "l1_ratio" : np.linspace(0,1,10).tolist(),
    "C": np.linspace(0,1,10).tolist(),
    "class_weight": [None, "balanced"]
}]

In [10]:
gs_logit = GridSearchCV(
    LogisticRegression(),
    param_grid=params,
    scoring = "f1",
    cv=5)

In [11]:
gs_logit.fit(X_train, y_train)

/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1197: UserWarning: l1_ratio 

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid=[{'C': [0.0, 0.1111111111111111, 0.2222222222222222,
                                0.3333333333333333, 0.4444444444444444,
                                0.5555555555555556, 0.6666666666666666,
                                0.7777777777777777, 0.8888888888888888, 1.0],
                          'class_weight': [None, 'balanced'],
                          'l1_ratio': [0.0, 0.1111111111111111,
                                       0.2222222222222222, 0.3333333333333333,
                                       0.4444444444444444, 0.5555555555555556,
                                       0.6666666666666666, 0.7777777777777777,
                                       0.8888888888888888, 1.0],
                          'penalty': [None, 'elasticnet']}],
             scoring='f1')

In [12]:
gs_logit.best_params_
gs_logit.score(X_train, y_train)

np.float64(0.9050385655614545)

In [13]:
gs_logit.score(X_test, y_test)

np.float64(0.8394546912590216)